<a href="https://colab.research.google.com/github/absolutemocha/sdaai/blob/main/20065320-Lim%20Zhao%20Hong-C2349C-AY2022CWF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep Learning CWF
## Lim Zhao Hong (20065320)

This dataset is a collection of MRI photos. The aim to apply image analysis and classify MRI images into four classes: Benign Tumor, Malignant Tumor, Pituitary Tumor and no Tumor.

## Import libraries

In [1]:
import keras, os
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications import vgg16
from keras import models
from keras import layers



In [2]:
classes=[]
filename='../input/brain-tumor-classification-mri'
for sub_folder in os.listdir(os.path.join(filename,'Training')):
    classes.append(sub_folder)
print(classes)



FileNotFoundError: ignored

In [ ]:
#import skimage.transform
import cv2
from keras.preprocessing.image import img_to_array

# Dimension of training data. If you are not feeding 224,224,3 into the VGG16, you will need to ensure 
# include_top=False when you import the VGG16 model (step 7.2.4)
IMAGE_DIMS = (64, 64, 3)
resized_train_images = []
resized_test_images = []

# Resize every image in the train and test dataset to intended dimension based on requirements
for image in train_images:
    image = cv2.resize(image, (IMAGE_DIMS[0], IMAGE_DIMS[1]))
    image = img_to_array(image)
    resized_train_images.append(image)
    
for image in test_images:
    image = cv2.resize(image, (IMAGE_DIMS[0], IMAGE_DIMS[1]))
    image = img_to_array(image)
    resized_test_images.append(image)

In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical

# Convert to array and normalize the RGB values
resized_train_images = np.array(resized_train_images, dtype="float32") / 255.0
resized_test_images = np.array(resized_test_images, dtype="float32") / 255.0

# One-hot encoding on the labels
train_labels_cat = to_categorical(train_labels)
test_labels_cat = to_categorical(test_labels)

Prepare the VGG16 model

In [ ]:
from keras import Model
from keras.applications import vgg16
from keras.layers import Dropout, Flatten, Dense


Load the VGG16 model without the top classification layer

In [ ]:
IMAGE_DIMS = (64, 64, 3)

base_model = vgg16.VGG16(weights='imagenet', 
                       include_top=False, 
                       input_shape=IMAGE_DIMS)

print(base_model.summary())

In [ ]:
for i, layer in enumerate(base_model.layers):
    print('Layer {}: {} ({})'.format(i, layer.name, layer.trainable))

"Freeze" base layers of VGG16

In [ ]:
for layer in base_model.layers:
     layer.trainable = False

In [ ]:
for i, layer in enumerate(base_model.layers):
    print('Layer {}: {} ({})'.format(i, layer.name, layer.trainable))

# Build classification layers
add our classication layers on top the base VGG16 base layers for our dataset

In [ ]:
from keras import models

#creating own layers in addition to VGG16 as the base layers
model = models.Sequential(base_model.layers)

In [ ]:
# afterr CNN layers, we need to flatten and create our fully connected layers and final output laters for classifications
# determine with you own estimation on the layers and nodes for your network. It does not need to be perfect, an estimation will work for a start
# you can modify the following to further tune your model

model.add(Flatten())
# you can uncomment the following dense layer to your preference of accuracy
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='softmax'))

print(model.summary())

In [ ]:
for i, layer in enumerate(model.layers):
    print('Layer {}: {} ({})'.format(i, layer.name, layer.trainable))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Perform training on our dataset

In [ ]:
# Modify accordingly if you want to fine-tune your model
history = model.fit(resized_train_images, train_labels_cat, validation_split=0.2, epochs=20, batch_size=32, verbose=1, shuffle=True)

test_loss, test_acc = model.evaluate(resized_test_images, test_labels_cat, verbose=1)
print('Test loss: {:.4f}'.format(test_loss))
print('Test accuracy: {:.4f}'.format(test_acc))

Evaluate the "transferred" and "re-trained" model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_loss(history):
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']
    x = list(range(1, len(test_loss) + 1))
    plt.plot(x, test_loss, color='red', label='Val loss')
    plt.plot(x, train_loss, label='Train loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss vs. Epoch')
    plt.legend()
    
plot_loss(history)

In [ ]:
def plot_accuracy(history):
    train_acc = history.history['accuracy']
    test_acc = history.history['val_accuracy']
    x = list(range(1, len(test_acc) + 1))
    plt.plot(x, test_acc, color='red', label='Val accuracy')
    plt.plot(x, train_acc, label='Train accuracy')  
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xlim([0, 20])
    plt.ylim([0, 100])
    plt.title('Accuracy vs. Epoch')  
    plt.legend(loc='lower right')

plot_accuracy(history)

# Perform Prediction
Predict an image by randomly choosing 1

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

imgSize = len(resized_test_images)
selected = random.randint(0, imgSize)
img = resized_test_images[selected]

# To create a mapping of the label class to allow display of the description instead of a number
categories = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

plt.imshow(img)
plt.axis('off')
class_index = np.where(test_labels_cat[selected] == 1)[0][0]
plt.title('Label: {}'.format(categories[class_index]))
plt.show()

In [ ]:
# Run the image through the deep neural network to make a prediction
image = np.expand_dims(img, axis=0)
prob = model.predict(image)    
    
print('Probability: {}\n'.format(prob[0]))
idx = np.argmax(prob)
predictions = categories[idx]
print('Predicted class: {} - {}'.format(idx, predictions))

Predict an image by loading an image from local directory

In [ ]:
# Modify and run this if you are running jupyter on your local computer
#file_path = os.path.join('sample_dataset', 'monkey_1.png')

## uncomment line 8 to line 11 and modify accordingly to your folder if you are running on Google Colab
from google.colab import drive
drive.mount('/content/drive')
colab_path = 'drive/My Drive/Colab Notebooks/SDAAI-C2349/LU08/sample_dataset/' # complete path in Google Drive as follows for file access
file_path = colab_path + "monkey_1.png"

In [ ]:
import cv2
import numpy as np
from keras.preprocessing import image

# Load the image that is in the same directory as the script
img2 = cv2.imread(file_path)

# Pre-process the image for classification
image2 = cv2.resize(img2, (IMAGE_DIMS[0], IMAGE_DIMS[1]))
image2 = image2.astype("float") / 255.0
image2 = img_to_array(image2)
image2 = np.expand_dims(image2, axis=0)

# Classify the input image
print("[INFO] classifying image...")
proba = model.predict(image2)
idx = np.argmax(proba)
predictions = categories[idx]
print('Predicted class: {} - {}'.format(idx, predictions))